* `coalesce` and `repartition` are functions on top of the dataframe. Do not get confused between **coalesce** on Data Frame and the coalesce function available to deal with null values in a given column.
* `coalesce` is typically used to **reduce number of partitions** to deal with as part of downstream processing. 
* `repartition` is used to reshuffle the data to **higher or lower number of partitions** to deal with as part of downstream partitioning.
* Make sure to use a cluster with higher configuration, if you would like to run and experience by your self.
  * 2 to 3 worker nodes using Standard with 14 to 16 GB RAM and 4 cores each.

In [0]:
df = spark.read.csv('dbfs:/databricks-datasets/asa/airlines', header=True)

In [0]:
help(df.coalesce)

Help on method coalesce in module pyspark.sql.dataframe:

coalesce(numPartitions) method of pyspark.sql.dataframe.DataFrame instance
 Returns a new :class:`DataFrame` that has exactly `numPartitions` partitions.
 
 Similar to coalesce defined on an :class:`RDD`, this operation results in a
 narrow dependency, e.g. if you go from 1000 partitions to 100 partitions,
 there will not be a shuffle, instead each of the 100 new partitions will
 claim 10 of the current partitions. If a larger number of partitions is requested,
 it will stay at the current number of partitions.
 
 However, if you're doing a drastic coalesce, e.g. to numPartitions = 1,
 this may result in your computation taking place on fewer nodes than
 you like (e.g. one node in the case of numPartitions = 1). To avoid this,
 you can call repartition(). This will add a shuffle step, but means the
 current upstream partitions will be executed in parallel (per whatever
 the current partitioning is).
 
 .. versionadded:: 1.4.0
 
 Parameters
 ----------
 numPartitions : int
 specify the target number of partitions
 
 Examples
 --------
 >>> df.coalesce(1).rdd.getNumPartitions()
 1

In [0]:
help(df.repartition)

Help on method repartition in module pyspark.sql.dataframe:

repartition(numPartitions, *cols) method of pyspark.sql.dataframe.DataFrame instance
 Returns a new :class:`DataFrame` partitioned by the given partitioning expressions. The
 resulting :class:`DataFrame` is hash partitioned.
 
 .. versionadded:: 1.3.0
 
 Parameters
 ----------
 numPartitions : int
 can be an int to specify the target number of partitions or a Column.
 If it is a Column, it will be used as the first partitioning column. If not specified,
 the default number of partitions is used.
 cols : str or :class:`Column`
 partitioning columns.
 
 .. versionchanged:: 1.6
 Added optional arguments to specify the partitioning columns. Also made numPartitions
 optional if partitioning columns are specified.
 
 Examples
 --------
 >>> df.repartition(10).rdd.getNumPartitions()
 10
 >>> data = df.union(df).repartition(5, "age")
 >>> data.show()
 +---+-----+
age| name|
 +---+-----+
 5| Bob|
 5| Bob|
 2|Alice|
 2|Alice|
 +---+-----+
 >>> data = data.repartition(7, "age")
 >>> data.show()
 +---+-----+
age| name|
 +---+-----+
 2|Alice|
 5| Bob|
 2|Alice|
 5| Bob|
 +---+-----+
 >>> data.rdd.getNumPartitions()
 7
 >>> data = data.repartition(3, "name", "age")
 >>> data.show()
 +---+-----+
age| name|
 +---+-----+
 5| Bob|
 5| Bob|
 2|Alice|
 2|Alice|
 +---+-----+

* `repartition` incurs **shuffling** and it takes time as data has to be shuffled to newer number of partitions.
* Also you can `repartition` the Data Frame based on specified columns.
* `coalesce` does not incur shuffling.
* We use `coalesce` quite often before writing the data to fewer number of files.

In [0]:
df = spark.read.csv('dbfs:/databricks-datasets/asa/airlines', header=True, inferSchema=True)

In [0]:
dbutils.fs.ls('dbfs:/databricks-datasets/asa/airlines')

Out[5]: [FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1987.csv', name='1987.csv', size=127162942),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1988.csv', name='1988.csv', size=501039472),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1989.csv', name='1989.csv', size=486518821),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1990.csv', name='1990.csv', size=509194687),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1991.csv', name='1991.csv', size=491210093),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1992.csv', name='1992.csv', size=492313731),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1993.csv', name='1993.csv', size=490753652),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1994.csv', name='1994.csv', size=501558665),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1995.csv', name='1995.csv', size=530751568),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1996.csv', name='1996.csv', size=533922363),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1997.csv', name='1997.csv', size=540347861),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1998.csv', name='1998.csv', size=538432875),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1999.csv', name='1999.csv', size=552926022),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2000.csv', name='2000.csv', size=570151613),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2001.csv', name='2001.csv', size=600411462),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2002.csv', name='2002.csv', size=530507013),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2003.csv', name='2003.csv', size=626745242),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2004.csv', name='2004.csv', size=669879113),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2005.csv', name='2005.csv', size=671027265),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2006.csv', name='2006.csv', size=672068096),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2007.csv', name='2007.csv', size=702878193),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/2008.csv', name='2008.csv', size=689413344)]

In [0]:
df.rdd.getNumPartitions()

Out[6]: 93

In [0]:
# coalescing the Dataframe to 16
df.coalesce(16).rdd.getNumPartitions()

Out[8]: 16

In [0]:
# not effective as coalesce can be used to reduce the number of partitioning.
# Faster as no shuffling is involved
df.coalesce(186).rdd.getNumPartitions()

Out[9]: 93

In [0]:
# incurs shuffling
# Watch the execution time and compare with coalesce
df.repartition(16).rdd.getNumPartitions()

Out[10]: 16

In [0]:
# repartitioned to higher number of partitions
df.repartition(186, 'Year', 'Month').rdd.getNumPartitions()

Out[11]: 186

One of the common usage of `coalesce` is to write to lesser number of files.

In [0]:
import getpass
username = getpass.getuser()

In [0]:

dbutils.fs.rm(f'/user/{username}/airlines', recurse=True)

Out[12]: False

In [0]:
df.write.mode('overwrite').csv(f'/user/{username}/airlines', header=True, compression='gzip')

In [0]:
dbutils.fs.ls(f'/user/{username}/airlines')

Out[15]: [FileInfo(path='dbfs:/user/root/airlines/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/user/root/airlines/_committed_3525485636792310121', name='_committed_3525485636792310121', size=8580),
 FileInfo(path='dbfs:/user/root/airlines/_started_3525485636792310121', name='_started_3525485636792310121', size=0),
 FileInfo(path='dbfs:/user/root/airlines/part-00000-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-281-1-c000.csv.gz', name='part-00000-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-281-1-c000.csv.gz', size=18371171),
 FileInfo(path='dbfs:/user/root/airlines/part-00001-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-282-1-c000.csv.gz', name='part-00001-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-282-1-c000.csv.gz', size=17653259),
 FileInfo(path='dbfs:/user/root/airlines/part-00002-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-283-1-c000.csv.gz', name='part-00002-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-283-1-c000.csv.gz', size=17653017),
 FileInfo(path='dbfs:/user/root/airlines/part-00003-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-284-1-c000.csv.gz', name='part-00003-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-284-1-c000.csv.gz', size=18572870),
 FileInfo(path='dbfs:/user/root/airlines/part-00004-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-285-1-c000.csv.gz', name='part-00004-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-285-1-c000.csv.gz', size=18329151),
 FileInfo(path='dbfs:/user/root/airlines/part-00005-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-286-1-c000.csv.gz', name='part-00005-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-286-1-c000.csv.gz', size=18221180),
 FileInfo(path='dbfs:/user/root/airlines/part-00006-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-287-1-c000.csv.gz', name='part-00006-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-287-1-c000.csv.gz', size=19064576),
 FileInfo(path='dbfs:/user/root/airlines/part-00007-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-288-1-c000.csv.gz', name='part-00007-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-288-1-c000.csv.gz', size=18500206),
 FileInfo(path='dbfs:/user/root/airlines/part-00008-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-289-1-c000.csv.gz', name='part-00008-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-289-1-c000.csv.gz', size=18605653),
 FileInfo(path='dbfs:/user/root/airlines/part-00009-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-290-1-c000.csv.gz', name='part-00009-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-290-1-c000.csv.gz', size=19122428),
 FileInfo(path='dbfs:/user/root/airlines/part-00010-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-291-1-c000.csv.gz', name='part-00010-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-291-1-c000.csv.gz', size=18532181),
 FileInfo(path='dbfs:/user/root/airlines/part-00011-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-292-1-c000.csv.gz', name='part-00011-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-292-1-c000.csv.gz', size=18249326),
 FileInfo(path='dbfs:/user/root/airlines/part-00012-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-293-1-c000.csv.gz', name='part-00012-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-293-1-c000.csv.gz', size=18807077),
 FileInfo(path='dbfs:/user/root/airlines/part-00013-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-294-1-c000.csv.gz', name='part-00013-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-294-1-c000.csv.gz', size=18518822),
 FileInfo(path='dbfs:/user/root/airlines/part-00014-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-295-1-c000.csv.gz', name='part-00014-tid-3525485636792310121-ab27bec0-3807-451e-8acb-065e4c55163f-2

In [0]:
df.repartition(16).write.mode('overwrite').csv(f'/user/{username}/airlines', header=True, compression='gzip')

In [0]:
dbutils.fs.ls(f'/user/{username}/airlines')

Out[17]: [FileInfo(path='dbfs:/user/root/airlines/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/user/root/airlines/_committed_3525485636792310121', name='_committed_3525485636792310121', size=8580),
 FileInfo(path='dbfs:/user/root/airlines/_committed_6109974810368428194', name='_committed_6109974810368428194', size=10062),
 FileInfo(path='dbfs:/user/root/airlines/_started_3525485636792310121', name='_started_3525485636792310121', size=0),
 FileInfo(path='dbfs:/user/root/airlines/_started_6109974810368428194', name='_started_6109974810368428194', size=0),
 FileInfo(path='dbfs:/user/root/airlines/part-00000-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-467-1-c000.csv.gz', name='part-00000-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-467-1-c000.csv.gz', size=238249407),
 FileInfo(path='dbfs:/user/root/airlines/part-00001-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-468-1-c000.csv.gz', name='part-00001-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-468-1-c000.csv.gz', size=238250823),
 FileInfo(path='dbfs:/user/root/airlines/part-00002-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-469-1-c000.csv.gz', name='part-00002-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-469-1-c000.csv.gz', size=238230879),
 FileInfo(path='dbfs:/user/root/airlines/part-00003-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-470-1-c000.csv.gz', name='part-00003-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-470-1-c000.csv.gz', size=238258202),
 FileInfo(path='dbfs:/user/root/airlines/part-00004-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-471-1-c000.csv.gz', name='part-00004-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-471-1-c000.csv.gz', size=238253382),
 FileInfo(path='dbfs:/user/root/airlines/part-00005-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-472-1-c000.csv.gz', name='part-00005-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-472-1-c000.csv.gz', size=238240101),
 FileInfo(path='dbfs:/user/root/airlines/part-00006-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-473-1-c000.csv.gz', name='part-00006-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-473-1-c000.csv.gz', size=238257624),
 FileInfo(path='dbfs:/user/root/airlines/part-00007-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-474-1-c000.csv.gz', name='part-00007-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-474-1-c000.csv.gz', size=238247402),
 FileInfo(path='dbfs:/user/root/airlines/part-00008-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-475-1-c000.csv.gz', name='part-00008-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-475-1-c000.csv.gz', size=238241451),
 FileInfo(path='dbfs:/user/root/airlines/part-00009-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-476-1-c000.csv.gz', name='part-00009-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-476-1-c000.csv.gz', size=238227582),
 FileInfo(path='dbfs:/user/root/airlines/part-00010-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-477-1-c000.csv.gz', name='part-00010-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-477-1-c000.csv.gz', size=238255038),
 FileInfo(path='dbfs:/user/root/airlines/part-00011-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-478-1-c000.csv.gz', name='part-00011-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-478-1-c000.csv.gz', size=238246321),
 FileInfo(path='dbfs:/user/root/airlines/part-00012-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-479-1-c000.csv.gz', name='part-00012-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-479-1-c000.csv.gz', size=238246573),
 FileInfo(path='dbfs:/user/root/airlines/part-00013-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-480-1-c000.csv.gz', name='part-00013-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21

In [0]:
# If you use repartition it will take longer time than this.
df.coalesce(16).write.mode('overwrite').csv(f'/user/{username}/airlines', header=True, compression='gzip')

File "<command-3977223084447808>" , line 2 
 df.coalesce(16).write.mode('overwrite').csv(f'/user/{username}/airlines', header=True, compression='gzip) 
 ^
 SyntaxError : EOL while scanning string literal

In [0]:
dbutils.fs.ls(f'/user/{username}/airlines')

Out[19]: [FileInfo(path='dbfs:/user/root/airlines/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/user/root/airlines/_committed_3525485636792310121', name='_committed_3525485636792310121', size=8580),
 FileInfo(path='dbfs:/user/root/airlines/_committed_6109974810368428194', name='_committed_6109974810368428194', size=10062),
 FileInfo(path='dbfs:/user/root/airlines/_started_3525485636792310121', name='_started_3525485636792310121', size=0),
 FileInfo(path='dbfs:/user/root/airlines/_started_6109974810368428194', name='_started_6109974810368428194', size=0),
 FileInfo(path='dbfs:/user/root/airlines/part-00000-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-467-1-c000.csv.gz', name='part-00000-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-467-1-c000.csv.gz', size=238249407),
 FileInfo(path='dbfs:/user/root/airlines/part-00001-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-468-1-c000.csv.gz', name='part-00001-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-468-1-c000.csv.gz', size=238250823),
 FileInfo(path='dbfs:/user/root/airlines/part-00002-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-469-1-c000.csv.gz', name='part-00002-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-469-1-c000.csv.gz', size=238230879),
 FileInfo(path='dbfs:/user/root/airlines/part-00003-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-470-1-c000.csv.gz', name='part-00003-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-470-1-c000.csv.gz', size=238258202),
 FileInfo(path='dbfs:/user/root/airlines/part-00004-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-471-1-c000.csv.gz', name='part-00004-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-471-1-c000.csv.gz', size=238253382),
 FileInfo(path='dbfs:/user/root/airlines/part-00005-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-472-1-c000.csv.gz', name='part-00005-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-472-1-c000.csv.gz', size=238240101),
 FileInfo(path='dbfs:/user/root/airlines/part-00006-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-473-1-c000.csv.gz', name='part-00006-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-473-1-c000.csv.gz', size=238257624),
 FileInfo(path='dbfs:/user/root/airlines/part-00007-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-474-1-c000.csv.gz', name='part-00007-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-474-1-c000.csv.gz', size=238247402),
 FileInfo(path='dbfs:/user/root/airlines/part-00008-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-475-1-c000.csv.gz', name='part-00008-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-475-1-c000.csv.gz', size=238241451),
 FileInfo(path='dbfs:/user/root/airlines/part-00009-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-476-1-c000.csv.gz', name='part-00009-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-476-1-c000.csv.gz', size=238227582),
 FileInfo(path='dbfs:/user/root/airlines/part-00010-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-477-1-c000.csv.gz', name='part-00010-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-477-1-c000.csv.gz', size=238255038),
 FileInfo(path='dbfs:/user/root/airlines/part-00011-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-478-1-c000.csv.gz', name='part-00011-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-478-1-c000.csv.gz', size=238246321),
 FileInfo(path='dbfs:/user/root/airlines/part-00012-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-479-1-c000.csv.gz', name='part-00012-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-479-1-c000.csv.gz', size=238246573),
 FileInfo(path='dbfs:/user/root/airlines/part-00013-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21c93df005-480-1-c000.csv.gz', name='part-00013-tid-6109974810368428194-fd8c08ff-7bef-43dc-9862-3c21